In [1]:
import numpy
import os
import pickle
import argparse
from numpy import argmax
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext.data as data
import torch.optim as optim
from torchtext.vocab import GloVe, Vocab
from torchtext.vocab import build_vocab_from_iterator
from sklearn.metrics import f1_score



In [2]:
# def read_file(filename):
#     with open(filename, "r") as file:
#         text = file.readlines()
#     return text

# def process_text(text):
#     X = []
#     Y = []
#     sentenceX = []
#     sentenceY = []
#     for line in text:
#         split = line.split(" ")
#         if len(split) > 1:
#             sentenceX.append(split[0])
#             sentenceY.append(split[1].replace("\n", ""))
#         else:
#             X.append(sentenceX)
#             Y.append(sentenceY)
#             sentenceX = []
#             sentenceY = []
#     return X, Y

# train_data = read_file("data/train.txt")
# test_data = read_file("data/test.txt")
# X1, Y1 = process_text(train_data)
# X2, Y2 = process_text(test_data)

# vocab = set([word.lower() for sentence in X1 for word in sentence])
# vocab.add("unk")
# vocab = list(vocab)
# num_words = len(vocab)
# num_tags = len(set([word.lower() for sentence in Y1 for word in sentence]))

# print("Total number of tagged sentences: {}".format(len(X1)))
# print("Vocabulary size: {}".format(num_words))
# print("Total number of tags: {}".format(num_tags))

# # text_field = data.Field(sequential=True)
# # dataset = data.Dataset(examples=vocab, fields=[('text', text_field), ...])
# # text_field.build_vocab(dataset)


Total number of tagged sentences: 8936
Vocabulary size: 17259
Total number of tags: 22


In [3]:
# for i in range(len(X1[1])):
#     print(X1[1][i], Y1[1][i])

Chancellor O
of B-PP
the B-NP
Exchequer I-NP
Nigel B-NP
Lawson I-NP
's B-NP
restated I-NP
commitment I-NP
to B-PP
a B-NP
firm I-NP
monetary I-NP
policy I-NP
has B-VP
helped I-VP
to I-VP
prevent I-VP
a B-NP
freefall I-NP
in B-PP
sterling B-NP
over B-PP
the B-NP
past I-NP
week I-NP
. O


In [4]:
# def encoding(sentence, to_idx):
#     idxs = torch.tensor([to_idx[w.lower()] for w in sentence], dtype=torch.long)
#     # for w in sentence:
#     #     try:
#     #         idxs.append(to_idx[w.lower])
#     #     except KeyError:
#     #         continue
#     # idxs = torch.tensor(idxs, dtype=torch.long)
#     return idxs


# word_to_idx = {}
# tag_to_idx = {}
# for i in range(len(X1)):
#     sentence = X1[i]
#     tags = Y1[i]

#     for word in sentence:
#         if word.lower() not in word_to_idx:
#             word_to_idx[word.lower()] = len(word_to_idx)
    
#     word_to_idx["unk"] = len(word_to_idx)

#     for tag in tags:
#         if tag.lower() not in tag_to_idx:
#             tag_to_idx[tag.lower()] = len(tag_to_idx)
# # print(word_to_idx)
# EMBEDDING_DIM = 100
# HIDDEN_DIM = 22

In [2]:
# class LSTMTagger(nn.Module):
    
#     def __init__(self, embedding_dim, hidden_dim, num_words, num_tags):
#         super(LSTMTagger, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.vocab = vocab

#         self.embeddings = nn.Embedding(num_words, embedding_dim)

#         # The LSTM takes word embeddings as inputs, and outputs hidden states
#         # with dimensionality hidden_dim.
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
#         # The linear layer that maps from hidden state space to tag space
#         self.hidden2tag = nn.Linear(hidden_dim, num_tags)
#     # def _init_embeddings(self):
#     #     glove = GloVe(name="6B", dim=300, cache="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\.vector_cache")
#     #     # TODO not initialize with zero for non existent vocab items in glove
#     #     self.embeddings.weight.data.copy_(glove.get_vecs_by_tokens(self.vocab.get_itos(), lower_case_backup=True))
        
#     def forward(self, sentence):
#         embeds = self.embeddings(sentence)
#         # print(embeds.shape)
#         lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
#         tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
#         tag_scores = F.log_softmax(tag_space, dim=1)
#         return tag_scores

In [3]:
# model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
# loss_function = nn.NLLLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1)

# # See what the scores are before training
# # Note that element i,j of the output is the score for tag j for word i.
# # Here we don't need to train, so the code is wrapped in torch.no_grad()
# # with torch.no_grad():
# #     inputs = encoding(X1[0], word_to_idx)
# #     tag_scores = model(inputs)
# #     print(tag_scores)

# for epoch in range(100):  # again, normally you would NOT do 300 epochs, it is toy data
#     # print(epoch)
#     for i in range(2):
#         sentence = X1[i]
#         tags = Y1[i]
#         # Step 1. Remember that Pytorch accumulates gradients.
#         # We need to clear them out before each instance
#         model.zero_grad()

#         # Step 2. Get our inputs ready for the network, that is, turn them into
#         # Tensors of word indices.
#         targets = encoding(tags, tag_to_idx)
#         # print(targets)
#         sentence_in = encoding(sentence, word_to_idx)
#         # print(sentence_in)
#         targets = encoding(tags, tag_to_idx)

#         # Step 3. Run our forward pass.
#         tag_scores = model(sentence_in)

#         # Step 4. Compute the loss, gradients, and update the parameters by
#         #  calling optimizer.step()
#         loss = loss_function(tag_scores, targets)
#         print(loss)
#         loss.backward()
#         optimizer.step()

# # See what the scores are after training
# with torch.no_grad():
#     inputs = encoding(X1[0], word_to_idx)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     print(tag_scores.shape)

NameError: name 'EMBEDDING_DIM' is not defined

In [7]:
# def get_scores(predY, trueY):

#     trueY_O = [i for i, x in enumerate(trueY) if x == "O"]
#     predY = [predY[i] for i in range(len(predY)) if i not in trueY_O]
#     trueY = [trueY[i] for i in range(len(trueY)) if i not in trueY_O]

#     print("Micro F1 score: ", f1_score(trueY, predY, average="micro"))
#     print("Macro F1 score: ", f1_score(trueY, predY, average="macro"))
#     print("Average F1 score: ", (f1_score(trueY, predY, average="micro") + f1_score(trueY, predY, average="macro")) / 2)



In [8]:
# #Testing the model
# tag_keys = list(tag_to_idx.keys())
# tag_values = list(tag_to_idx.values())
# sentence_in = X1[1]
# true_Y = Y1[1]
# with torch.no_grad():
#     inputs = encoding(sentence_in, word_to_idx)
#     tag_scores = model(inputs)
# pred_Y = [tag_keys[position].upper() for position in [tag_values.index(argmax(y)) for y in tag_scores]]
# print(true_Y)
# print(pred_Y)



# get_scores(pred_Y, true_Y)


['O', 'B-PP', 'B-NP', 'I-NP', 'B-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'O']
['O', 'B-PP', 'B-NP', 'I-NP', 'B-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'O']
Micro F1 score:  0.96
Macro F1 score:  0.7714285714285715
Average F1 score:  0.8657142857142857


In [4]:
class ELL881Dataset(torch.utils.data.Dataset):
    def __init__(self, vocab=None, data_dir=None, file_path=None, split="train") -> None:
        super().__init__()
        self.data_dir = data_dir
        self.split=split

        if file_path is not None:
            self.file_path = file_path
        else:
            self.file_path = os.path.join(data_dir, f"{split}.txt")
        self.text = self.read_file(self.file_path)
        self.X, self.Y = self.process_text(self.text)
        self.vocab = build_vocab_from_iterator(self.X, specials=["<unk>"], min_freq=1)
        self.tags = build_vocab_from_iterator(self.Y, min_freq=1)
        self.vocab.set_default_index(self.vocab["<unk>"])

    
    def __len__(self):
        return len(self.X)
        
    def read_file(self, filename):
        with open(filename, "r") as file:
            text = file.readlines()
        return text
    
    def process_text(self, text):
        X = []
        Y = []
        sentenceX = []
        sentenceY = []
        for line in text:
            split = line.split(" ")
            if len(split) > 1:
                sentenceX.append(split[0])
                sentenceY.append(split[1].replace("\n", ""))
            else:
                X.append(sentenceX)
                Y.append(sentenceY)
                sentenceX = []
                sentenceY = []
        return X, Y
    
    def tokenize_sent(self, sentence):
        return torch.tensor(self.vocab(sentence)).long()
    
    def tokenize_tags(self, targets):
        return torch.tensor(self.tags(targets)).long()
    


    
train_dataset = ELL881Dataset(data_dir="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\data")
test_dataset = ELL881Dataset(data_dir="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\data", split="test")
print(train_dataset.tokenize_sent(train_dataset.X[0]))
print(train_dataset.X[0])
print(train_dataset.Y[0])
print(train_dataset.tokenize_tags(train_dataset.Y[0]))
print("Total number of tagged sentences: {}".format(len(train_dataset)))
print("Vocabulary size: {}".format(train_dataset.vocab.__len__()))
print("Total number of tags: {}".format(train_dataset.tags.__len__()))
    
    
            

tensor([11555,     8,     2,  1774,    16,  1163,   176,     5,   211,   316,
         1215,  6115,   107,   649,   596,    10,   456,     1,   248,    10,
         2625,  2962,     1,  4919,     5,   601,     6,  1400,  1751,    20,
          731,     7,   542,     9,  8901, 14950,     3])
['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.']
['B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O']
tensor([1, 4, 1, 0, 3, 5, 5, 5, 5, 1, 0, 0, 7, 1, 0, 4, 1, 2, 8, 4, 1, 1, 2, 3,
        

In [5]:
for i in range(len(train_dataset.X[0])):
    print(train_dataset.X[0][i], train_dataset.Y[0][i])

Confidence B-NP
in B-PP
the B-NP
pound I-NP
is B-VP
widely I-VP
expected I-VP
to I-VP
take I-VP
another B-NP
sharp I-NP
dive I-NP
if B-SBAR
trade B-NP
figures I-NP
for B-PP
September B-NP
, O
due B-ADJP
for B-PP
release B-NP
tomorrow B-NP
, O
fail B-VP
to I-VP
show I-VP
a B-NP
substantial I-NP
improvement I-NP
from B-PP
July B-NP
and I-NP
August I-NP
's B-NP
near-record I-NP
deficits I-NP
. O


In [6]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size, vocab : Vocab) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.vocab = vocab
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self._init_embeddings()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.loss_func = nn.NLLLoss()

    def _init_embeddings(self):
        glove = GloVe(name="6B",dim=self.embedding_dim,cache="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\.vector_cache",)
        self.embeddings.weight.data.copy_(glove.get_vecs_by_tokens(self.vocab.get_itos(), lower_case_backup=True))
        self.embeddings.weight.requires_grad = False
    
    def forward(self, sentence, targets=None):
        embedding = self.embeddings(sentence)
        out, _ = self.lstm(embedding.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        if targets is not None:
            loss = self.loss_func(tag_scores, targets)
            return loss, tag_scores
        return tag_scores






    


In [35]:
# # @dataclass
# # class Params:
    
# #     eval_steps: int = 100
# #     save_steps: int = 100
# #     logging_steps: int = 50
# #     warmup_steps: int = 0
# #     # learning_rate: float = 0.1
# #     lr_scheduler_type: str = "linear"
# #     sgd_epsilon: float = 1e-8
# #     weight_decay: int = 0
# #     save_total_limit: int = 5
# #     eval_accumulation_steps: int = 5
# #     gradient_accumulation_steps: int = 1
# #     early_stopping_patience: int = 5
# #     early_stopping_threshold: float = 0.001
# #     train_batch_size: int = 32
# #     eval_batch_size: int = 32
# #     prediction_loss_only: bool = False


# #     # output_dir: str = "models/"
# #     # do_train: bool = False
# #     # do_eval: bool = True
# #     # do_predict: bool = False
# #     # prediction_file_path: str = "data/test.txt"
# #     resume_from_checkpoint: bool = False
# #     # num_train_epochs: int = 10
# #     run_name: str = "test"

# #     model_init_dir: str = None
# #     # data_dir: str = "data"
# #     do_freeze: bool = True
# #     force_same_lr: bool = False

# parser = argparse.ArgumentParser()
# parser.add_argument("--eta", default=0.1, type=float, help="Learning Rate")
# parser.add_argument("--epochs", default=10, type=int, help="Number of training epochs")
# parser.add_argument("--do_train", default=True, type=bool, help="Whether Training has to be performed or not")
# parser.add_argument("--do_predict", default=False, type=bool, help="")
# parser.add_argument("--do_eval", default=False, type=bool, help="")
# parser.add_argument("--output_dir", default="models/", type=str, help="")
# parser.add_argument("--pred_file_path", default="data/test.txt", type=str, help="")
# parser.add_argument("--data_dir", default="data", type=str, help="")
# parser.add_argument("--embedding_size", default=300, type=int, help="")
# parser.add_argument("--hidden_dim", default=128, type=int, help="")
# # parser.add_argument()
# # parser.add_argument()
# # parser.add_argument()
# # parser.add_argument()
# # parser.add_argument()
# args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--eta ETA] [--epochs EPOCHS]
                             [--do_train DO_TRAIN] [--do_predict DO_PREDICT]
                             [--do_eval DO_EVAL] [--output_dir OUTPUT_DIR]
                             [--pred_file_path PRED_FILE_PATH]
                             [--data_dir DATA_DIR]
                             [--embedding_size EMBEDDING_SIZE]
                             [--hidden_dim HIDDEN_DIM]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"8166658a-429f-4f89-b0e9-7f32d4b8aa05" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\admin\AppData\Roaming\jupyter\runtime\kernel-v2-7900Nlk7djXY0Utl.json


SystemExit: 2

In [11]:
class Trainer:
    def __init__(self, model, train_dataset, eval_dataset, optimizer) -> None:
        # self.args = args
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = eval_dataset
        self.optim = optimizer
        # for p in self.model.parameters():
        #     if p.requires_grad == True:
        #         print(p.shape)     

    def train(self):


        for epoch in range(10):
            print(epoch)
            for i in range(len(train_dataset)):
                sentence = self.train_dataset.X[i]
                tags = self.train_dataset.Y[i]
                
                # Step 1. Remember that Pytorch accumulates gradients.

                self.model.zero_grad()

                # Step 2. Get our inputs ready for the network, that is, turn them into
                # Tensors of word indices.
                inputs = self.train_dataset.tokenize_sent(sentence)
                targets = self.train_dataset.tokenize_tags(tags)

                # Step 3. Run our forward pass and get loss values.
                loss, tag_scores = self.model(inputs, targets)
                # Step 4. Compute the gradients, and update the parameters by
                #  calling optimizer.step()
                loss.backward()

                nn.utils.clip_grad_value_(self.model.parameters(), clip_value=1.0)
                optimizer.step()
    
    def predict(self, dataset : ELL881Dataset):
        output = []
        for i in range(len(dataset)):
            sentence = dataset.X[i]

            self.model.zero_grad()
            input = train_dataset.tokenize_sent(sentence)
            tag_scores = self.model.forward(input)
            pred = [train_dataset.tags.lookup_token(torch.argmax(y)) for y in tag_scores]
            output += pred 

        return output

            
            
    def evaluate(self, trueY, predY):
        trueY_O = [i for i, x in enumerate(trueY) if x == "O"]
        predY = [predY[i] for i in range(len(predY)) if i not in trueY_O]
        trueY = [trueY[i] for i in range(len(trueY)) if i not in trueY_O]

        print("Micro F1 score: ", f1_score(trueY, predY, average="micro"))
        print("Macro F1 score: ", f1_score(trueY, predY, average="macro"))
        print("Average F1 score: ", (f1_score(trueY, predY, average="micro") + f1_score(trueY, predY, average="macro")) / 2)

    # def save_model(): 

In [28]:
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
model1 = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, train_dataset.vocab.__len__(), train_dataset.tags.__len__(), vocab=train_dataset.vocab)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.1)
# for p in model.parameters():
#     if p.requires_grad == True:
#         print(p.shape)
# print(model.parameters())

In [29]:
    
train_dataset = ELL881Dataset(data_dir="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\data")
test_dataset = ELL881Dataset(data_dir="E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\data", split="test")
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, train_dataset.vocab.__len__(), train_dataset.tags.__len__(), vocab=train_dataset.vocab)
trainer = Trainer(model=model1, train_dataset=train_dataset, eval_dataset=test_dataset, optimizer=optimizer)
trainer.train()

torch.save(model.state_dict(), "E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\models\pytorch_model.bin")



0
1
2
3
4
5
6
7
8
9


In [59]:

# model1 = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, train_dataset.vocab.__len__(), train_dataset.tags.__len__(), vocab=train_dataset.vocab)
# model1.load_state_dict(torch.load("E:\IIT DELHI\SEMESTER-8\ELL881\ASSIGNMENTS\A2\models\pytorch_model.bin"))



In [32]:

# trainer = Trainer(model=model1, train_dataset=train_dataset, eval_dataset=test_dataset, optimizer=optimizer)
predY = trainer.predict(train_dataset)

print(predY[0:10])

trueY = []
for i in range(len(train_dataset)):
    for tag in train_dataset.Y[i]:
        trueY.append(tag)
print(trueY[0:10])
trainer.evaluate(trueY, predY)

['B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP']
['B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP']
Micro F1 score:  0.9585584115327077
Macro F1 score:  0.5339874427631582
Average F1 score:  0.7462729271479329
